Importação bibliotecas

In [1]:
import pandas as pd # Fundamental para análise e manipulação de dados
import requests # Para fazer requisições HTTP (acessar APIs, baixar dados da web)
import os # Para interagir com o sistema operacional (manipular arquivos, pastas)
from datetime import datetime # Módulo para trabalhar com datas e horas
import yfinance as yf # Baixar dados do mercado financeiro (Yahoo Finance)
from pytrends.request import TrendReq # Extrair dados de tendências de busca (Google Trends)
from sidrapy import get_table # Função específica para buscar dados de tabelas do SIDRA (IBGE)

Função de Coleta do Banco Central (SGS)

- Busca os indicadores e salva em um arquivo CSV na pasta data_raw

In [ ]:
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y/%m/%d')
data_raw_path = '../data_raw/'

# Se existir a pasta ok, se não é criada
os.makedirs(data_raw_path, exist_ok=True)

# Definição {dicionário} de indicadores BCB
indicadores_bcb = {
    '432': 'selic_meta',             # SELIC (Meta) - Mensal
    '433': 'ipca_mensal',            # IPCA (Var. % mensal) - Mensal
    '4393': 'icc_consumidor_mensal', # Índice de Confiança do Consumidor - Mensal
    '24364': 'pib_mensal'            # Produto interno bruto - Mensal
}

def dados_bc (codigo_indicador, nome_indicador):

    start_date_fmt = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
    end_date_fmt = datetime.now().strftime('%d/%m/%Y')

    url_api = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_indicador}/dados?formato=json&dataInicial={start_date_fmt}&dataFinal={end_date_fmt}"
    
    #Chamar a API
    response = requests.get(url_api)
    
    if response.status_code == 200: #Código 200 significa êxito
        dados_json = response.json()
        
        if not dados_json: # Check se a API retornar sem dados
            print(f"   > AVISO: Série {nome_indicador} não retornou dados.")
            return

        # Converte para DataFrame
        df = pd.DataFrame(dados_json)
        
        # Renomeia a coluna 'valor' para o nome do indicador
        df = df.rename(columns={'valor': nome_indicador})
        
        # Salvar o arquivo CSV individual (Criando novas pastas)
        file_path = os.path.join(data_raw_path, f'{nome_indicador}.csv')
        df.to_csv(file_path, index=False)
        
        print(f"   > Sucesso. Salvo em: {file_path}")
        
    else:
        print(f"   > ERRO! Falha ao buscar {nome_indicador}. Status: {response.status_code}")

# Loop para a função de coleta BCB
for codigo, nome in indicadores_bcb.items():
    dados_bc(codigo, nome)

   > Sucesso. Salvo em: ../data_raw/selic_meta.csv
   > Sucesso. Salvo em: ../data_raw/ipca_mensal.csv
   > Sucesso. Salvo em: ../data_raw/icc_consumidor_mensal.csv
   > Sucesso. Salvo em: ../data_raw/pib_mensal.csv


Coleta Taxa de Desocupação (IBGE/SIDRA)

In [13]:
try:
    df_pmc = get_table(
        table_code='4095',
        territorial_level='1',
        ibge_territorial_code='all',
        variable='4099',
        period='all' 
    )
    
    if not df_pmc.empty:
        df_pmc = df_pmc.loc[1:, ['V', 'D2C']]
        df_pmc.columns = ['taxa_desocupacao', 'periodo_codigo_trimestre']
        df_pmc['taxa_desocupacao'] = pd.to_numeric(df_pmc['taxa_desocupacao'], errors='coerce')
        # Filtro de data
        df_pmc = df_pmc[df_pmc['periodo_codigo_trimestre'] >= '201904']
        
        file_path = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
        df_pmc.to_csv(file_path, index=False)
        print(f"   > Sucesso. Salvo em: {file_path}")
    else:
        print("   > AVISO: Dados de desemprego não retornaram.")
        
except Exception as e:
    print(f"   > ERRO GERAL! Falha ao buscar Taxa de Desocupação : {e}")

   > Sucesso. Salvo em: ../data_raw/desemprego_trimestral.csv


Coleta Google Trends para as empresas

In [5]:
end_date = datetime.now().strftime('%Y-%m-%d')

pytrends = TrendReq(hl='pt-BR', tz=360)

keywords = ['Magazine Luiza', 'Casas Bahia', 'Mercado Livre']

pytrends.build_payload(keywords, timeframe=f'2020-01-01 {end_date}',geo='BR')

df = pytrends.interest_over_time()

if not df.empty:
    if 'isPartial' in df.columns:
        df = df.drop(columns=['isPartial'])

    file_path = os.path.join(data_raw_path, 'gtrends.csv')
    df.to_csv(file_path, index_label='data')
    print(f"   > Sucesso. Salvo Google Trends em: {file_path}")
else:
    print("   > AVISO: Dados do Google Trends não retornaram.")

   > Sucesso. Salvo Google Trends em: ../data_raw/gtrends.csv


c:\Users\Thiag\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Coleta Pesquisa mensal comércio

In [6]:
start_date = '2020-01-01'

url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.1455/dados?formato=json"
df = pd.read_json(url)

df['data'] = pd.to_datetime(df['data'], dayfirst=True)
df = df[df['data'] >= start_date]

df.rename(columns={'valor' : 'indice_PMC'},inplace=True)

df.reset_index(drop=True,inplace=True)

file_path = os.path.join(data_raw_path, 'pmc.csv')
df.to_csv(file_path, index=False)

print(df.tail())


         data  indice_PMC
62 2025-03-01      104.42
63 2025-04-01      104.11
64 2025-05-01      108.39
65 2025-06-01      101.99
66 2025-07-01      106.58
